# Import the libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import swish
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2

# Load and preprocess dataset

In [ ]:
def load_dataset(data_dir, img_size=(32, 32)):
    classes = ['Normal', 'Reversal', 'Corrected']
    X, y = [], []
    
    for split in ['Train', 'Test']:
        split_path = os.path.join(data_dir, split)
        for label, class_name in enumerate(classes):
            class_path = os.path.join(split_path, class_name)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
                img = cv2.resize(img, img_size)  # Resize to 32x32
                img = img / 255.0  # Normalize pixel values to [0,1]
                X.append(img)
                y.append(label)
    
    X = np.array(X).reshape(-1, 32, 32, 1)  # Reshape for CNN input
    y = to_categorical(y, num_classes=len(classes))  # Convert labels to one-hot encoding
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Modified LeNet-5 Model

In [ ]:
def build_model():
    model = Sequential([
        Conv2D(6, (5, 5), activation=swish, padding='same', input_shape=(32, 32, 1)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(16, (5, 5), activation=swish, padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(120, (5, 5), activation=swish, padding='same'),
        BatchNormalization(),
        Dropout(0.1),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(84, activation=swish),
        Dropout(0.1),
        Dense(3, activation='softmax')  # Output layer for 3 classes
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

# Load dataset 

In [ ]:
data_directory = "Gambo"
X_train, X_test, y_train, y_test = load_dataset(data_directory)

# Instantiate and train the model

In [ ]:
model = build_model()
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Plot training history

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()

plot_history(history)

# Print model summary

In [ ]:
model.summary()